# Start Active Players: Automating My Daily Lineups
Automatically setting my NBA Fantasy lineups by Jaume Clave

December 19, 2020

I've been playing Fantasy NBA for 5 years now. Before the real NBA season starts we get a group
of friends to join our 12 team, 9cat h2h NBA Fantasy league on Yahoo. It's good fun to build out
a team and see how players perform throughout the year.

It can get quite competitive between the team owners and leagues in the past have been won on
small margins. Each owner tries to get as many advantages as they can. People set up Twitter
alerts so that they are first to know about an injury or trade. Team gamble on *sleepers* in hope
 that a waiver wire add can explode into a huge signing.

One of the most commons ways of losing a h2h match up is forgetting to set your lineups.
Unfortunately, if you leave one of your players on the bench, you miss out on all the points that
 player makes for you. This often happens as this is something that needs to be done and checked
 daily. Life gets in the way and one forgets.

This project aims to fix this. This project creates a script using Selenium that logs into my
Yahoo Fantasy and automatically sets my players as *active*. Meaning their in-game performances
will count towards my totals. This script will be executed daily using Windows Task Scheduler.

## Index

[Selenium](#Selenium)


## Selenium
Selenium WebDriver is one of the most popular tools for Web UI Automation. The selenium package
is  used to automate web browser interaction from Python. Web UI Automation refers to the
automatic  execution of the actions preformed in a web browser window like navigating to a
website, filing forms that deal with input to text boxes, with radio buttons, downloading of
images  and files, submission of forms and general website interactions. With Selenium a user is
 able to automate all this stuff; automate the boring stuff!

In order to automate with selenium on Google Chrome, chromedriver.exe must be downloaded because
a  path to this needs to be executable on your computer. WebDriver is an open source tool for
automated testing of webapps across many browsers. It provides capabilities for navigating to web
  pages, user input, JavaScript execution, and more. ChromeDriver is a standalone server that
  implements the W3C WebDriver standard.

This section will use Selenium to automate the three steps involved in setting a daily lineups:

    1. Authenticate on Yahoo and navigate to the "My Team" page
    2. Find the "Start Active Players" button and click it
    3. Find the "Start" button in the popup and click it

Each step will be broken down and visualised.

In [7]:
# Import modules
from selenium import webdriver
from selenium.webdriver import Chrome
import time

### ChromeDriver

In [8]:
# Chrome arguments
CHROME_USER_DATA_DIRECTORY_ARGUMENT = r"user-data-dir=C:\Users\Jaume\AppData\Local\Google\Chrome" \
                                      r"\User Data\Python"
CHROME_DISABLE_NOTIFICATION_ARGUMENT = "--disable-notifications"
CHROME_EXECUTABLE_PATH =  r"C:\Users\Jaume\Documents\Python " \
                          r"Projects\yahoo_fantasy_lineups\chromedriver.exe"

In [9]:
# Load Profile
options = webdriver.ChromeOptions()

# Load Chrome profile and download directory
options.add_argument(CHROME_USER_DATA_DIRECTORY_ARGUMENT)
options.add_argument(CHROME_DISABLE_NOTIFICATION_ARGUMENT)

# Chrome with profile
driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)
time.sleep(2)

<ipython-input-9-1fcbf30b4fb1>:9: DeprecationWarning: use options instead of chrome_options
  driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)


In [10]:
# Load "My Team" page
MY_TEAM_URL = "https://basketball.fantasysports.yahoo.com/nba/55374/6"

# Get URL
driver.get(MY_TEAM_URL)
time.sleep(1)

<img src="http://drive.google.com/uc?export=view&id=1NNjQc_r7MsA4gj7Nca3atwOI_rNiCXoJ" />

### Finding Elements With Selenium
There are many ways to find an element on a page using selenium, all require some basic
understanding of HTML. XPath is the language used for locating nodes in an XML document. As HTML
 can be an implementation of XML (XHTML), we can leverage this powerful language to target
 elements  in the WhatsApp Web application. XPath extends beyond (as well as supporting) the
 simple  methods of locating by id or name attributes, and opens up all sorts of new
 possibilities  such as locating the third checkbox on the page.

One of the main reasons for using XPath is when you don’t have a suitable id or name attribute
for  the element you wish to locate. You can use XPath to either locate the element in absolute
terms (not advised), or relative to an element that does have an id or name attribute. XPath
locators can also be used to specify elements via attributes other than id and name.

Absolute XPaths contain the location of all elements from the root (HTML) and as a result are
likely to fail with only the slightest adjustment to the application. By finding a nearby element
  with an id or name attribute (ideally a parent element) you can locate your target element
  based  on the relationship. This is much less likely to change and can make your tests more
  robust.

The first element that needs to be found is the button that contains the "Start Active Players"
text. We can find the XPATH by inspecting the element. This button, when clicked, loads a popup
which we will need to perform another action on. Once the initial button is clicked, we delay the
execution of a system slightly to allow for new elements to load and populate the HTML.

In [11]:
# Click "Start Active Players" button
START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT = "//*[(text()='Start Active Players')]"

# Find and click
start_active_players_button = driver.find_element_by_xpath(START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
start_active_players_button.click()
time.sleep(1)

<img src="http://drive.google.com/uc?export=view&id=166BW20jWjch4RVLEI_5gCFDbz64_Vc8a" />

In [12]:
# Click "Rest of Week Start" button
REST_OF_WEEK_START_BUTTON_XPATH_TEXT = "//*[(text()='Start')]"

# Find and click
start_rest_of_week_button = driver.find_element_by_xpath(REST_OF_WEEK_START_BUTTON_XPATH_TEXT)
start_rest_of_week_button.click()

<img src="http://drive.google.com/uc?export=view&id=1gaAK0Ea9OmGc5wIuiWll7FvHk3dO0leT" />

Following the successful player activation message the driver and window are closed. The Python
logic works and this process must now be optimised so that it can automatically be executed daily
 using Windows Task Scheduler.

## Python Script

In [36]:
# Sleep time (seconds)

def open_yahoo_my_team():
    """
    Function opens the Yahoo "My Team" web page. ChromeDriver options are loaded which facilitate
    the Yahoo login. The driver is returned.
    :return driver: ChromeDriver used to open Yahoo session
    """
    # Load Profile
    options = webdriver.ChromeOptions()

    # Load Chrome profile and download directory
    options.add_argument(CHROME_USER_DATA_DIRECTORY_ARGUMENT)
    options.add_argument(CHROME_DISABLE_NOTIFICATION_ARGUMENT)

    # Chrome with profile
    driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)

    driver.get(MY_TEAM_URL)
    time.sleep(SLEEP_SECONDS)

    return driver

def click_button_yahoo_my_team(driver, button_xpath_text):
    """
    Function finds a webelement by XPATH and automatically clicks it.
    :param driver: ChromeDriver used to open Yahoo session
    :param button_xpath_text: XPATH of desired element
    :return driver: ChromeDriver used to open Yahoo session
    """
    webelement_button = driver.find_element_by_xpath(button_xpath_text)
    webelement_button.click()
    time.sleep(SLEEP_SECONDS)

    return driver

In [40]:
# Combining the functions
def set_daily_yahoo_lineups():
    """
    Function ties the entire process. It first logins to Yahoo via a ChromeDriver. It then finds
    the buttons required to be clicked to activate starting players.
    :return: None
    """
    driver = open_yahoo_my_team()
    click_button_yahoo_my_team(driver, START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
    click_button_yahoo_my_team(driver, REST_OF_WEEK_START_BUTTON_XPATH_TEXT)

    return None

# Testing the function
set_daily_yahoo_lineups()

<ipython-input-36-b2b85705d337>:17: DeprecationWarning: use options instead of chrome_options
  driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)


### Windows Task Scheduler
Task Scheduler is a component of Microsoft Windows that provides the ability to schedule the
launch  of programs or scripts at pre-defined times or after specified time intervals: job
scheduling. The tools used to create and run virtually any task automatically. Typically, the
system and certain apps use the scheduler to automate maintenance tasks (such as disk
defragmentation, disk cleanup, and updates), but anyone user can run it. With this experience,
can  start applications, run commands, and execute scripts at a particular day and time, or
you can also trigger tasks when a specific event occurs.

Task Scheduler works by keeping tabs of the time and events on your computer and executes the
task as soon as the condition is met. This script will be executed at 11am each day.

### if \_\_name\_\_ == "\_\_main\_\_":


In [42]:
# Putting it all together
CHROME_USER_DATA_DIRECTORY_ARGUMENT = r"user-data-dir=C:\Users\Jaume\AppData\Local\Google\Chrome" \
                                      r"\User Data\Python"
CHROME_DISABLE_NOTIFICATION_ARGUMENT = "--disable-notifications"
CHROME_EXECUTABLE_PATH =  r"C:\Users\Jaume\Documents\Python " \
                          r"Projects\yahoo_fantasy_lineups\chromedriver.exe"

MY_TEAM_URL = "https://basketball.fantasysports.yahoo.com/nba/55374/6"

SLEEP_SECONDS = 1

START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT = "//*[(text()='Start Active Players')]"
REST_OF_WEEK_START_BUTTON_XPATH_TEXT = "//*[(text()='Start')]"


def open_yahoo_my_team():
    """
    Function opens the Yahoo "My Team" web page. ChromeDriver options are loaded which facilitate
    the Yahoo login. The driver is returned.
    :return driver: ChromeDriver used to open Yahoo session
    """
    # Load Profile
    options = webdriver.ChromeOptions()

    # Load Chrome profile and download directory
    options.add_argument(CHROME_USER_DATA_DIRECTORY_ARGUMENT)
    options.add_argument(CHROME_DISABLE_NOTIFICATION_ARGUMENT)

    # Chrome with profile
    driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)

    driver.get(MY_TEAM_URL)
    time.sleep(SLEEP_SECONDS)

    return driver


def click_button_yahoo_my_team(driver, button_xpath_text):
    """
    Function finds a webelement by XPATH and automatically clicks it.
    :param driver: ChromeDriver used to open Yahoo session
    :param button_xpath_text: XPATH of desired element
    :return driver: ChromeDriver used to open Yahoo session
    """
    webelement_button = driver.find_element_by_xpath(button_xpath_text)
    webelement_button.click()
    time.sleep(SLEEP_SECONDS)

    return driver


def set_daily_yahoo_lineups():
    """
    Function ties the entire process. It first logins to Yahoo via a ChromeDriver. It then finds
    the buttons required to be clicked to activate starting players.
    :return: None
    """
    driver = open_yahoo_my_team()
    click_button_yahoo_my_team(driver, START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
    click_button_yahoo_my_team(driver, REST_OF_WEEK_START_BUTTON_XPATH_TEXT)

    return None


if __name__ == "__main__":
    driver = open_yahoo_my_team()
    click_button_yahoo_my_team(driver, START_ACTIVE_PLAYERS_BUTTON_XPATH_TEXT)
    click_button_yahoo_my_team(driver, REST_OF_WEEK_START_BUTTON_XPATH_TEXT)


<ipython-input-42-1ad3cae6affb>:28: DeprecationWarning: use options instead of chrome_options
  driver = Chrome(CHROME_EXECUTABLE_PATH, chrome_options = options)
